In [2]:
# !pip install gnss_lib_py|

In [3]:
import numpy as np
import gnss_lib_py as glp
import matplotlib.pyplot as plt
from datetime import datetime, timezone
import pandas as pd

https://drive.google.com/drive/folders/17vNrS_-Kq0msv2r9VHDkry3aEAxUcdAS

### Pseudorange computation

In [4]:
def compute_pseudoranges(raw_data):
    nanosecs_in_week = 604800e9
    speed_of_light = 299792458
    week_no = np.floor(-raw_data['FullBiasNanos']/nanosecs_in_week)
    corrected_rx_time = raw_data['TimeNanos'] + raw_data['TimeOffsetNanos']
    adjusted_rx_bias = raw_data['FullBiasNanos'] + raw_data['BiasNanos']
    rx_time_gps_frame = corrected_rx_time - adjusted_rx_bias
    rx_time_week_frame = rx_time_gps_frame - week_no * nanosecs_in_week
    rho_nanosecs = rx_time_week_frame - raw_data['ReceivedSvTimeNanos']
    rho_m = rho_nanosecs * speed_of_light/1e9
    return rho_m

In [5]:
rover_path = './Oval/RoundLoop1_gnss_log_2024_11_17_16_52_31.txt'
base_path = './Oval/RoundLoop1_BaseStation_gnss_log_2024_11_17_16_52_30.txt'

In [6]:
rover_data = glp.AndroidRawGnss(input_path=rover_path,
                              filter_measurements=True,
                              measurement_filters={"sv_time_uncertainty" : 500.},
                              verbose=True)

base_data = glp.AndroidRawGnss(input_path=base_path,
                              filter_measurements=True,
                              measurement_filters={"sv_time_uncertainty" : 500.},
                              verbose=True)

rover_rho = compute_pseudoranges(rover_data)
base_rho = compute_pseudoranges(base_data)

sv_time_uncertainty removed 1
sv_time_uncertainty removed 1


### Pull the ephemeris (don't work)

In [7]:
rover_gps_millis_start, rover_gps_millis_end = rover_data['gps_millis'][0], rover_data['gps_millis'][-1] 
base_gps_millis_start, base_gps_millis_end = base_data['gps_millis'][0], base_data['gps_millis'][-1]

In [8]:
rover_ephemeris_file = glp.load_ephemeris(file_type="sp3",
                              gps_millis=np.array([rover_gps_millis_start, rover_gps_millis_start]),
                              verbose=True)

base_ephemeris_file = glp.load_ephemeris(file_type="sp3",
                              gps_millis=np.array([base_gps_millis_start, base_gps_millis_start]),
                              verbose=True)

ephemeris dates needed: [datetime.date(2024, 11, 17), datetime.date(2024, 11, 18)]
FTP downloading /gnss/products/2341/GFZ0MGXRAP_20243220000_01D_05M_ORB.SP3.gz from gdc.cddis.eosdis.nasa.gov
FTP downloading /gnss/products/2341/GFZ0MGXRAP_20243230000_01D_05M_ORB.SP3.gz from gdc.cddis.eosdis.nasa.gov
550 Failed to open file. Failed to retrieve /gnss/products/2341/GFZ0MGXRAP_20243230000_01D_05M_ORB.SP3.gz from gdc.cddis.eosdis.nasa.gov
using previously downloaded file:
 /Users/thomashuang/school/AA 272/final/dgnss_aa272/data/ephemeris/sp3/GFZ0MGXRAP_20243220000_01D_05M_ORB.SP3
FTP downloading /gnss/products/2341/GFZ0MGXRAP_20243230000_01D_05M_ORB.SP3.gz from gdc.cddis.eosdis.nasa.gov
using previously downloaded file:
 /Users/thomashuang/school/AA 272/final/dgnss_aa272/data/ephemeris/sp3/GFZ0MGXRAP_20243220000_01D_05M_ORB.SP3
FTP downloading /gnss/products/2341/GFZ0MGXRAP_20243230000_01D_05M_ORB.SP3.gz from gdc.cddis.eosdis.nasa.gov


error_perm: 550 Failed to open file. Failed to retrieve /gnss/products/2341/GFZ0MGXRAP_20243230000_01D_05M_ORB.SP3.gz from gdc.cddis.eosdis.nasa.gov

### Calculate true ranges

In [9]:
BASE_STATION_LOCATION_1 = np.array([37.429711,-122.169381,25]).reshape(3, 1)
BASE_STATION_LOCATION_2 = np.array([37.4290631,-122.1723136,25]).reshape(3, 1) # multipath test

DURAND = np.array([[37.426906], [-122.173278], [21.010]]) 

In [10]:
## Load the RinexNav data for that day
rinex_nav_paths = glp.load_ephemeris(
    "rinex_nav",
    np.array(np.min(base_data['gps_millis']),np.max(base_data['gps_millis'])),
    verbose=True,
)
rinex_nav = glp.RinexNav(rinex_nav_paths, verbose=True)

ephemeris dates needed: [datetime.date(2024, 11, 17), datetime.date(2024, 11, 18)]
using previously downloaded file:
 /Users/thomashuang/school/AA 272/final/dgnss_aa272/data/ephemeris/rinex/nav/BRDM00DLR_S_20243220000_01D_MN.rnx
using previously downloaded file:
 /Users/thomashuang/school/AA 272/final/dgnss_aa272/data/ephemeris/rinex/nav/BRDM00DLR_S_20243230000_01D_MN.rnx
using previously downloaded file:
 /Users/thomashuang/school/AA 272/final/dgnss_aa272/data/ephemeris/rinex/nav/BRDM00DLR_S_20243220000_01D_MN.rnx
using previously downloaded file:
 /Users/thomashuang/school/AA 272/final/dgnss_aa272/data/ephemeris/rinex/nav/BRDM00DLR_S_20243230000_01D_MN.rnx


In [11]:
def find_sv_pos(rinex, sv_id, gnss_id, gps_millis):
    """ finding sv position with ephemeris data FOR DURAND receiver"""
    n = len(sv_id)
    position_info = np.zeros((n, 3))
    for i in range(n):
        sv_at_millis = glp.find_sv_states(gps_millis[i], rinex)
        cropped_sv_at_millis = sv_at_millis.where('sv_id', sv_id[i], 'eq')
        gnss_id_at_millis = cropped_sv_at_millis.where('gnss_id', gnss_id[i], 'eq')
        position_info[i, 0] = np.mean(gnss_id_at_millis['x_sv_m'])
        position_info[i, 1] = np.mean(gnss_id_at_millis['y_sv_m'])
        position_info[i, 2] = np.mean(gnss_id_at_millis['z_sv_m'])
    return position_info

# FOR DURAND DATA
# def get_measured_pseudorange(df,sv_id):
#     """ get measured pseudorange of specific sv for durand receiver"""
#     filtered_df = df[df.iloc[:,4] == str(sv_id)]
#     return filtered_df.iloc[:,14]

def distance_to_groundtruth(sv_pos, ground_truth):
    """ calculate ground truth distance from SV to Durand receiver"""
    gt_ecef = glp.geodetic_to_ecef(ground_truth)
    distance = np.linalg.norm(sv_pos[:, :, None] - gt_ecef[None, :, :], axis=1)
    return distance

def correction(true_range, base_pseudorange, rover_pseudorange):
    """ true_range: ground truth distance from distance_to_groundtruth function
    base_pseudorange: pseudorange for base station
    pseudorange: pseudorange for phone """
    e = true_range - base_pseudorange
    adjusted_pseudorange = rover_pseudorange + e
    return adjusted_pseudorange

# def get_pseudorange(raw_data, sv_id):
#     """ get measure pseudorange of the rover with specified sv_id"""
#     cropped_data = raw_data.where("sv_id", sv_id, "eq")
#     pseudorange = compute_pseudoranges(cropped_data)
#     return pseudorange

In [12]:

sv_pos = find_sv_pos(rinex_nav, base_data['sv_id'], base_data['gnss_id'], base_data['gps_millis'])

# sv_pos = find_sv_pos(rover_ephemeris, 6)
# true_ranges = distance_to_groundtruth()

In [40]:
rover_data['sv_id']

array([14,  6, 16, ..., 29, 30, 34])

In [39]:
base_data['sv_id']

array([21, 14, 15, ..., 30, 27, 15])

### Actual dGNSS shit

In [28]:
length = 1000
base_pseudorange = compute_pseudoranges(base_data)
true_range = distance_to_groundtruth(sv_pos, BASE_STATION_LOCATION_1).flatten()
rover_pseudorange = compute_pseudoranges(rover_data)
corrected_pseudorange = correction(true_range[:length], base_pseudorange[:length], rover_pseudorange[:length])

## are we matching rover and base SVs

In [29]:
# Define four arrays

x = sv_pos[:length, 0]
y = sv_pos[:length, 1]
z = sv_pos[:length, 2]

# Create DataFrame using the arrays
df = pd.DataFrame({
    'x_sv_m': x,
    'y_sv_m': y,
    'z_sv_m': z,
    'corr_pr_m': corrected_pseudorange
})

In [30]:
import numpy as np
import pandas as pd

def newton_raphson(x0, bu_0, max_iterations, dataset):
    """
    Function description: perform newton raphson to compute position of observer based on available satellite
    data, stop function when we have updates in delta_x, delta_bu that are less than a centimeter,
    or hit maximum number of iterations

    Inputs:
    1) x_0: initial guess for position
    2) bu_0: initial guess for clock bias
    3) max_iterations: maximum number of iterations until covergence
    4) dataset_name: name of file to read in

    return: 
    1) optimized position
    2) optimized clock bias
    3) number of iterations until covergence
    """
    x_est = x0
    b_est = bu_0


    for i in range(max_iterations):

        # Get Geometry Matrix and deltas for rho
        geometry_matrix = []
        delta_rho = []
        for index, sat in dataset.iterrows():
            geometry_matrix.append(get_geometry_row(x_est, sat['x_sv_m'], sat['y_sv_m'], sat['z_sv_m']))
            delta_rho.append(sat['corr_pr_m'] - get_theoretical_pseudoranges(x_est, b_est, sat['x_sv_m'], sat['y_sv_m'], sat['z_sv_m']))

        # Convert into matricies to make it easier to use numpy functions
        geometry_matrix = np.array(geometry_matrix)
        delta_rho = np.array(delta_rho)

        # Update the deltas
        delta_x, delta_bu = newton_raphson_step(geometry_matrix, delta_rho)
        x_est = np.add(x_est, delta_x)
        b_est = b_est + delta_bu
    
        if True: #i%10 == 0:
            print("round: ",i+1)
            print("x_est: ", x_est)
            print("b_est: ", b_est)

        if delta_x[0] < 0.01 and delta_x[1] < 0.01 and delta_x[2] < 0.01 and delta_bu < 0.01:
            break
    
    
    print("round: ",i+1)
    print("FINAL - x_est: ", x_est)
    print("FINAL - b_est: ", b_est)
    return x_est


def get_geometry_row(x_est, x_sv_m, y_sv_m, z_sv_m):
    """
    Function description: Compute each row for the geometry matrix given a satellite position and
    current guess for position estimate

    Inputs:
    1) x_est: current guess for current position estimate 
    2) x_sv_m: a set for satellite position x
    3) y_sv_m: a set for satellite position y
    4) z_sv_m: a set for satellite position z


    return: 
    1) Geometry Matrix G (first 3 columns unit vector)
    """

    x = x_sv_m - x_est[0]
    y = y_sv_m - x_est[1]
    z = z_sv_m - x_est[2]

    magnitude = np.sqrt((x)**2 + (y)**2 + (z)**2)

    return ([-x/magnitude, -y/magnitude, -z/magnitude, 1])



def get_theoretical_pseudoranges(x_est, b_est, x_sv_m, y_sv_m, z_sv_m):
    """
    Function description: Compute each row for the geometry matrix given a satellite position and
    current guess for position estimate

    Inputs:
    1) x_est: current position estimate 
    2) b_est: current clock bias estimate 
    3) x_sv_m: a set for satellite position x
    4) y_sv_m: a set for satellite position y
    5) z_sv_m: a set for satellite position z

    return: 
    1) Theoretical pseudorange measurement vector
    """


    x = x_sv_m - x_est[0]
    y = y_sv_m - x_est[1]
    z = z_sv_m - x_est[2]

    magnitude = np.sqrt((x)**2 + (y)**2 + (z)**2)
    b_k = 0 # assume its zero for all satellites, already corrrected for in data

    return magnitude + b_est - b_k

def newton_raphson_step(G,delta_rho):
    """
    Function description: Take one step of newton-raphson optimization, compute update of 
    delta_x and delta_bu

    Input:
    1) G: geometry matrix
    2) delta_rho: difference of theoretical and measured pseudoranges 

    return:
    1) delta_x
    2) delta_bu
    """
    G_t = G.transpose()
    deltas = np.matmul(np.matmul(np.linalg.inv(np.matmul(G_t, G)),G_t),delta_rho)

    delta_x = deltas[0:3]
    delta_bu = deltas[3]

    return delta_x, delta_bu

In [31]:
import gnss_lib_py as glp

df_cleaned = df.dropna()
x = newton_raphson([0,0,0],0,30,df_cleaned)

print("Lat/Long Version: ",glp.ecef_to_geodetic(x))

round:  1
x_est:  [ 9.69815083e+11  1.82384120e+12 -7.57204176e+11]
b_est:  187164142251963.2
round:  2
x_est:  [1.93722383e+16 6.47340936e+15 2.68771248e+16]
b_est:  -4467100321213837.0
round:  3
x_est:  [ 1.31483078e+18 -1.71487911e+19  1.23149643e+18]
b_est:  1.5669694300238362e+18
round:  4
x_est:  [ 8.70003991e+19 -4.53212313e+18  1.73409259e+20]
b_est:  -1.4937827578343617e+19
round:  5
x_est:  [-1.39352886e+20 -1.40070997e+22 -7.71198075e+20]
b_est:  3.3526678375448733e+20
round:  6
x_est:  [ 6.41764076e+23 -1.54992867e+22  1.29278774e+23]
b_est:  -3.901728967357948e+21
round:  7
x_est:  [ 4.14880894e+25  2.14872904e+27 -1.24951355e+24]
b_est:  1.4194671528009324e+25
round:  8
x_est:  [-5.19156409e+29 -1.24537118e+28 -2.49215115e+30]
b_est:  1.9484115803893706e+28
round:  9
x_est:  [ 7.17634135e+30  1.03118425e+33 -6.46014219e+30]
b_est:  4.403316495587776e+29
round:  10
x_est:  [-5.87980013e+34  8.61925922e+32  4.48711754e+34]
b_est:  -1.6277305210167027e+32
round:  11
x_est:  